In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import random
import time

With this notebook, we will analyze ramachandran plots from GROMACS and compare them with database RP. Dihedrals are divided in four sets: alanine, glycine, proline and all the others.

First of all, we have the functions that creates a dictionary in which each key is an amino acid and each value is a dataframe. The columns of this dataframe are the $\phi$ and $\psi$ dihedrals.
The database files are in the 'database_csv' folder, while the GROMACS xvg file is in the 'input' folder.

We considered four class: 'ALA', 'GLY', 'PRO', 'ALL' (which contains all the other amino acids). The whole notebook is based on data organised as follows:

{'ALL': {'ARG':dataframe_arg, 'ASN':dataframe_asn, ...}}

There are 4 different notebook, one for each class.


NOTE: all the functions were designated for daset like

{'ALL': {'ARG':dataframe_arg,

             'ASN':dataframe_asn, ...},

  'ALA': {'ALA':dataframe_ala},

  'GLY': {'GLY':dataframe_gly},
 
  'PRO': {'PRO':dataframe_pro}}

In [2]:
def make_database():#aminos):
    #aminos is taken as input so that only the "interesting" aminoacid are considered.
    aminos = ['ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'HIS', 'ILE', 'LEU', 'LYS', 'PHE', 'SER', 'THR', 'TRP', 'TYR', 'VAL']
    
    db_dict = {}

    for amino in aminos:
        path = 'database_csv/%s.csv' % amino
        df = pd.read_csv(path)
        df.columns = ['aa', 'phi', 'psi']
        
        df = df.drop('aa', axis =1)
        df['weight'] = 1 /len(df.index) #normalized
        db_dict[amino] = df
    
    return db_dict #IT'S GOOD, DON'T TOUCH

####################################################################################################

def read_rama(path):
    #firstly, we read the .xvg file and make a proper pandas dataframe
    data = open(path, 'r')
    rama = pd.read_csv(data, sep = '\s+')
    rama.columns = ['phi', 'psi', 'aa']

    #in rama.xvg data are organized as \phi \psi aa-num so we have to split the last index
    rama[['type', 'num']] = rama['aa'].str.split('-', 1, expand=True)

    #values sorted by amino acid so that computations can be faster to implement
    rama = rama.drop(['aa', 'num'], axis = 1).sort_values('type').reset_index(drop=True)

    aminos = rama['type'].unique()

    rama_dict = {amino: rama[rama['type'] == amino].drop('type', axis = 1).reset_index() for amino in aminos}
    
    del rama_dict['ALA']
    del rama_dict['GLY']
    del rama_dict['PRO']

    return {'ALL':rama_dict} #.sort_values(['row', 'col'])

####################################################################################################

def separate(dictionary):
    
    ala_dict = {'ALA':dictionary['ALA']}
    gly_dict = {'GLY':dictionary['GLY']}
    pro_dict = {'PRO':dictionary['PRO']}
    del dictionary['ALA']
    del dictionary['GLY']
    del dictionary['PRO']

    return {'ALL':dictionary, 'ALA':ala_dict, 'GLY':gly_dict, 'PRO':pro_dict}

We have the $\phi$ and $\psi$ dihedrals. GROMACS defines two different potential term for each dihedral. They differ from each other for:
- the constant $k_i\geq 0$
- the phase angle $\theta_i \in [-\pi, \pi]$
- the multiplicity $m_i$, an integer $\in [1,6]$

To perform the reweighting, we will use the GROMACS potential given a dihedral $\theta$:
$$V_{i}(\theta)=k_{i}(1+\cos(m_i\theta - \theta_i))$$

The values of constant and phase are randomly generated with the random.random() function. While, for multiplicity, we will modify only a single value at a time. The starting array of multiplicities will be the one of the previous best guess.

In [3]:
def randomizer(mult):

    dih = {'ALL': random_dihedral()}
    const = {'ALL': random_constant(5)}
    
    for amino in mult.keys():
        mult[amino][random.randrange(0,len(mult))] = random.randrange(1,7)
    
    return dih, const, mult


def random_dihedral():

    return [360 * random.random() - 180 for i in range(4)]


def random_constant(limit):

    return [limit * random.random() for i in range(4)]

####################################################################################################

def make_weights(rama_dict, dih, const, mult, beta):

    for amino in rama_dict.keys():
        rama = rama_dict.get(amino)
        rama = rama.assign(weight=lambda x: weight(x.phi, x.psi, dih, const, mult, beta))
    
        rama['weight'] = rama['weight'] / np.sum(rama['weight'])
        rama_dict[amino] = rama
        #print(rama)
        
    return rama_dict



def weight(phi, psi, dih, const, mult, beta):
    
    d2r = np.pi / 180
    
    gd_42 = (const[0] * ( 1 + np.cos(mult[0] * (psi * d2r) - (dih[0] * d2r))))
    gd_43 = (const[1] * ( 1 + np.cos(mult[1] * (phi * d2r) - (dih[1] * d2r))))        
    gd_44 = (const[2] * ( 1 + np.cos(mult[2] * (phi * d2r) - (dih[2] * d2r))))
    gd_45 = (const[3] * ( 1 + np.cos(mult[3] * (psi * d2r) - (dih[3] * d2r))))

    return np.exp(-1 * beta * (gd_42 + gd_43 + gd_44 + gd_45))

The make_matrix function 'translate' our RP in a $180\times180$ matrix, i.e. in a two-dimensional normalized histogram. Each entry of the matrix is the height of the relative bin. We will make a matrix from the db (only once, it is given as input) and one from the test data everytime we generate new random values.

We have two ways to evaluate our reweighting.

Firstly, we analyze its efficiency, i.e. how many bins are "thrown away". Given weight $w_i$ of the i-th bin, we define it as:
$$E = \frac{\left( \sum_{bins} w_i \right)}{\sum_{bins} (w_i)^2}$$
We want this efficiency to be greater of a certain value.

Then, we build the score function considering the weight of each bin in database $\delta_i$ and from the reweight $\gamma_i$:
$$ S = \sum_{amino acids} \left[ \frac{\sum_{i} (\delta_i - \gamma_i)^2}{\sum_{i} (\delta_i)^2} \right]$$

In [4]:
def make_matrix(rama_dict):

    mat_dict = {}

    for amino in rama_dict.keys():
        df = rama_dict.get(amino)
        mat, x1, x2 = np.histogram2d(df['psi'], df['phi'], bins=180, weights=df['weight'], density=True)
        mat_dict[amino] = mat
        #print(len(mat))
    
    return mat_dict

####################################################################################################

def efficiency_analysis(rama_dict):
    
    perc = []

    for amino in rama_dict.keys():
        df = rama_dict.get(amino)
        s_weight = np.power(np.sum(df['weight']), 2)
        s2_weight = np.sum(np.power(df['weight'], 2))

        perc.append(s_weight/(s2_weight*len(df.index)))
    #print(perc)
    return perc

####################################################################################################

def score_comp(db_mat, rama_dict):
    
    mats_rama = make_matrix(rama_dict)
    scores = []

    for amino in mats_rama.keys():

        mat_db = db_mat.get(amino)
        mat_rama = mats_rama.get(amino)
    
        mat_diff = np.power(mat_db - mat_rama, 2)
        mat_sq = np.power(mat_db, 2)

        num = np.sum(mat_diff)
        den = np.sum(mat_sq)

        scores.append(num/den)

    return np.sum(scores)



The output writers function just write out in four different files the data that gave us good scores. The order of data is fixed:
- the score
- the ratio of acceptance (i.e how many trials were accepted over the total)
- the efficiency
- $\theta_i$, $k_i$, $m_i$

In [5]:
def all_writer(score_ref, ratio, i, temp, acceptance, score, perc, dih, const,mult):
    
    score_ref, ratio = writer(score, score_ref, i, perc, temp, acceptance, ratio, dih, const, mult)

    return score_ref, ratio

####################################################################################################

def writer(score, score_ref, i, perc, temp, acceptance, ratio, dih, const, mult):

    for amino in ratio.keys():
        path = 'metropolis_run/score_%s.txt' % amino 
        
        ref_score = score_ref[amino]
        sco = score[amino]
        
        rat = np.int(ratio[amino])
        if sco < ref_score:
            rat = rat + 1
            with open(path, 'a') as file:
                print('%f %f %f %f %f %f %f %f %f %f %f %f %f %f %f' %(score[amino], rat/(i+1), perc[amino], dih[amino][0], dih[amino][1], dih[amino][2], dih[amino][3], const[amino][0], const[amino][1], const[amino][2], const[amino][3], mult[amino][0], mult[amino][1], mult[amino][2], mult[amino][3]), file=file)
            score_ref[amino] = score[amino]
            ratio[amino] = rat
            
        else:
            if np.exp(-(score[amino] - score_ref[amino]) / temp[amino]) > acceptance[amino]:
                rat = rat + 1
                with open(path, 'a') as file:
                    print('%f %f %f %f %f %f %f %f %f %f %f %f %f %f %f' %(score[amino], rat/(i+1), perc[amino], dih[amino][0], dih[amino][1], dih[amino][2], dih[amino][3], const[amino][0], const[amino][1], const[amino][2], const[amino][3], mult[amino][0], mult[amino][1], mult[amino][2], mult[amino][3]), file=file)
                ratio[amino] = rat
    return score_ref, ratio

As input, two dictionaries are required:
- a dictionary of matrices from the database
- a dictionary of dataframes from the simulation ramachandran plot
 This means that we will create RV for all the quantities of interest.  


In [7]:
mat_db = {'ALL':make_matrix(make_database())}
rama_dict = read_rama('input/rama_diff_2.xvg')

We base this notebook on Metropolis algorithm.
This means that we will create random variables for each quantity of interest. Then we will compute the potential terms and the relative reweighting of the histogram.

To accept the new move, it has to satisfy some parameters:
- it has to be efficient, $E\leq E_{ref}$, where $E_{ref}$ is chosen in the main.
- its score must be lower than the best available score $S \leq S_{best}$.

In case it is efficient but the score is not smaller, it can still be accepted. We can define a Monte Carlo temperature $\tau_{mc}$ and generate a random acceptance $\alpha$. Given the current score $S$ and the best score $S_{best}$ , if
$$ \exp\left[-\frac{S-S_{best}}{\tau}\right] \geq \alpha$$
holds, then the move is accepted.

If the score is accepted, it becomes the new reference.

The number of moves is decided in the for cycle.

In [11]:
def main(mat_db, rama_dict):
    
    score_ref = {'ALL':17.228867}
    ratio = {'ALL':0}
    for key in ratio.keys():
        ratio[key] = np.int(ratio[key])
    mult = {'ALL':[6,2,2,1]}

    #Monte Carlo temperature
    temp = {'ALL':0.06}

    start_time = time.time()

    #mat_0 = separate(mat_db)
    #mat_1 = separate(rama_dict)
    #print(mat_1)

    for i in range(25000):
        #start_time = time.time()
        
        #print(mat_db.keys())
        acceptance = {'ALL':random.random()}
        
        dih, const, mult = randomizer(mult)
        
        score, perc = metropolis(mat_db, rama_dict, dih, const, mult, beta=1/2.4943389)
        
        score_ref, ratio = all_writer(score_ref, ratio, i, temp, acceptance, score, perc, dih, const, mult)

    print('Metropolis done in %s seconds!' %(time.time() - start_time))

    return 'Done!'

####################################################################################################

def metropolis(mat_db, rama_dict, dih, const, mult, beta):
    
    
    for amino in dih.keys():
        rama_dict[amino] = make_weights(rama_dict[amino], dih[amino], const[amino], mult[amino], beta)
        #print(type(rama_dict['GLY']))
    
    
    perc = {'ALL':np.array(efficiency_analysis(rama_dict['ALL'])).mean()}
    #print(perc)
    score = {}

    for amino in dih.keys():
        if  np.float(perc[amino]) >= 0.65:
            score[amino] = score_comp(mat_db[amino], rama_dict[amino])
        else:
            score[amino] = 1000

    return score, perc


In [12]:
main(mat_db, rama_dict)

Metropolis done in 17775.53225684166 seconds!


'Done!'